In [1]:
import pandas as pd
import numpy as np

In [2]:
d = pd.read_table('checkins.dat', sep = '|', skipinitialspace=True)

c:\users\dmitry\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.
c:\users\dmitry\appdata\local\programs\python\python37-32\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
d.head(5)

,id,user_id,venue_id,latitude,longitude,created_at
0,---------+---------+----------+---------------...,NaN,NaN,NaN,NaN,NaN
1,984301,2041916.0,5222.0,NaN,NaN,2012-04-21 17:39:01
2,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
3,984315,1764391.0,5222.0,NaN,NaN,2012-04-21 17:37:18
4,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43


In [4]:
d = d.dropna()

In [5]:
d.head(5)

,id,user_id,venue_id,latitude,longitude,created_at
2,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
4,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
8,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-21 17:39:22
10,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
11,984232,93870.0,380645.0,33.448377,-112.074037,2012-04-21 17:38:18


In [6]:
d = d.iloc[:100000]

In [7]:
d.shape

(100000, 6)

In [8]:
from sklearn.cluster import MeanShift

In [9]:
d.columns

Index(['id    ', 'user_id ', 'venue_id ', 'latitude      ', 'longitude     ',
       'created_at      '],
      dtype='object')

In [10]:
d.columns = d.columns.str.strip()

In [11]:
d.columns

Index(['id', 'user_id', 'venue_id', 'latitude', 'longitude', 'created_at'], dtype='object')

In [12]:
d = d.loc[:, 'latitude':'longitude']

In [13]:
d.head(5)

,latitude,longitude
2,38.895112,-77.036366
4,33.800745,-84.410520
8,45.523452,-122.676207
10,40.764462,-111.904565
11,33.448377,-112.074037


In [14]:
ms = MeanShift(bandwidth=0.1)
ms.fit(d)
labels = ms.labels_
cluster_centers = ms.cluster_centers_

In [15]:
labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)

In [16]:
print("number of estimated clusters : %d" % n_clusters_)

number of estimated clusters : 3231


In [17]:
ms.cluster_centers_

array([[  40.7177164 ,  -73.99183542],
       [  33.44943805, -112.00213969],
       [  33.44638027, -111.90188756],
       ...,
       [ -37.8229826 ,  145.1811902 ],
       [ -41.2924945 ,  174.7732353 ],
       [ -45.0311622 ,  168.6626435 ]])

In [18]:
labels

array([ 5,  7, 30, ..., 25, 19,  4], dtype=int32)

In [19]:
d['labels'] = labels

In [20]:
d.head(5)

,latitude,longitude,labels
2,38.895112,-77.036366,5
4,33.800745,-84.410520,7
8,45.523452,-122.676207,30
10,40.764462,-111.904565,65
11,33.448377,-112.074037,1


In [21]:
data_sorted = d.groupby('labels').count()

In [22]:
data_sorted.head()

,latitude,longitude
labels,,
0,12506,12506
1,4692,4692
2,3994,3994
3,3363,3363
4,3526,3526


In [24]:
clusters = data_sorted[data_sorted.latitude > 15].reset_index().labels.values

In [26]:
centers = []
for i in clusters:
    centers.append(ms.cluster_centers_[i])

In [29]:
offices = np.array([[33.751277, -118.188740],
                    [25.867736, -80.324116],
                    [51.503016, -0.075479],
                    [52.378894, 4.885084],
                    [39.366487, 117.036146],
                    [-33.868457, 151.205134]])

In [30]:
def distance(point1, point2):
    return ((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)**0.5

In [32]:
answer_index = 0
min_dist = 0
i = 0
while i < len(centers):
    distances = [distance(xx, centers[i]) for xx in offices]
    if min_dist == 0:
        min_dist = min(distances)
        answer_index = i
    else:
        if min_dist > min(distances):
            min_dist = min(distances)
            answer_index = i
    i += 1

In [33]:
answer_index

417

In [34]:
min_dist

0.007834758163107856

In [36]:
centers[answer_index]

array([-33.86063043, 151.20477593])

In [37]:
def write_answer(center):
    with open("answer.txt", "w") as f:
        f.write(str(center[0]) + ' ' + str(center[1]))

In [39]:
write_answer(centers[answer_index])